In [1]:
import re

# Texto extraído del OCR
ocr_text_2 = """
Invoice\nswitch\tDallas, TX 75267-4592\nPO Box 674592\t\t\t\t\t\t\t\tPage 1 of 1\n\n\tInvoice Date Due Date\tInvoice No.\n\t09/22/23\t08/27/24\t1556267\n\nToni Hackel\n6600 Kalanianaole Hwy Ste 200,\nHonolulu\n\nAccount No.\t\t\tP.O. Number\n\tServices for month of March\nU-08805490\t\t\t\tPO-2OPC-4316\nDescription\t\t\t\tQuantity\tRate\tAmount\nTransport | Switch Fiber Pair (Intra-campus) | Pairs (4419693704) (04/2023|10 Gbps Fiber\t2,912.98\t934.09\t2,720,985.49\nto HOEpyb (YSPG4VFH) (04/2023)\nCarrier Taxes for Transport | 230 Gbps Wavelength Diverse between Sparks, OR 56789 and\t3,500.87\t6,229.33\t21,808,074.52\nPlano, NV 98765 (SNpTfT) (NJYM5MQP) (07/2023 Taxes) (07/2023)\nSpecial Partnership Discount (03/2023)\t\t\t\t\t8.00\t-561.00\t-4,488.00\nItem Discount: Special Partnership Discount (11/2023)\t\t\t\t8.00\t-296.00\t-2,368.00\nTransport | 82 Gbps Fiber to YDDTJOrnuW (L0KSKNCB) (01/2024)\t\t\t5,607.34\t656.07\t3,678,807.55\nCarrier Taxes for Transport | 45 Gbps Wavelength Diverse between Hillsboro, NV 98765\t3,949.46\t7,128.95\t28,155,502.87\nand Portland, TX 56789 (a488ZH) (8D3PKG77) (05/2023 Taxes) (05/2023)\nTransport | 97 Gbps Fiber to 3XMOyFdB (5EE7PIWY) (02/2024)\t\t\t942.71\t1,918.01\t1,808,127.21\n\n\tTotal USD\t$58,164,641.64\n\nPlease update your system with our new remittance instruction located on this invoice. Please contact accounts receivable@switch.com with any\nquestions.\n\nPlease make payments to: Switch, Ltd.\nWire/ACH Payment: PNC Bank, N.A.\tAccount No.:3802512896 ACH Routing:083009060 Wire Routing: 043000096 SWIFT:PNCCUS33\nFor questions, please call or email Accounts Receivable Department.\nPhone No.\tFax No.\t\t\tE-Mail\t\t\t\t\tWeb Site\n702.522.2940\t702.444.9546\taccountsreceivable@switch.com\t\t\twww.switch.com\nPlease remit payment for amount due. If payment submitted is different from amount due, a detailed statement of where to apply the funds must be included\nwith payment."""

# Paso 1: Extraer solo la sección de la lista de precios
match = re.search(r"Description\t+\t+Quantity\t+Rate\t+Amount\n(.*?)\n\n\tTotal", ocr_text_2, re.DOTALL)
if match:
    prices_section = match.group(1)
else:
    prices_section = ""

# Paso 2: Unir líneas partidas correctamente
lines = prices_section.split("\n")
merged_lines = []
buffer = ""

for line in lines:
    if re.search(r"\t\d{1,3}(,\d{3})*\.\d{2}\t\d{1,3}(,\d{3})*\.\d{2}\t\d{1,3}(,\d{3})*\.\d{2}", line):
        if buffer:
            merged_lines.append(buffer.strip())  # Agregar línea completa
        buffer = line
    else:
        buffer += " " + line.strip()  # Concatenar con la anterior

if buffer:
    merged_lines.append(buffer.strip())  # Última línea

# Paso 3: Extraer los datos con regex
pattern = re.compile(r"(.+?)\t+([\d,]+\.?\d*)\t+([\d,]+\.?\d*)\t+([\d,]+\.?\d*)")
items = pattern.findall("\n".join(merged_lines))

# Imprimir resultados
print("Items Vendidos:")
for item in items:
    description, quantity, rate, amount = item
    print(f"Descripción: {description.strip()}\nCantidad: {quantity}\nRate: {rate}\nAmount: {amount}\n")


Items Vendidos:
Descripción: Transport | Switch Fiber Pair (Intra-campus) | Pairs (4419693704) (04/2023|10 Gbps Fiber
Cantidad: 2,912.98
Rate: 934.09
Amount: 2,720,985.49

Descripción: Carrier Taxes for Transport | 230 Gbps Wavelength Diverse between Sparks, OR 56789 and
Cantidad: 3,500.87
Rate: 6,229.33
Amount: 21,808,074.52

Descripción: Transport | 82 Gbps Fiber to YDDTJOrnuW (L0KSKNCB) (01/2024)
Cantidad: 5,607.34
Rate: 656.07
Amount: 3,678,807.55

Descripción: Carrier Taxes for Transport | 45 Gbps Wavelength Diverse between Hillsboro, NV 98765
Cantidad: 3,949.46
Rate: 7,128.95
Amount: 28,155,502.87

Descripción: Transport | 97 Gbps Fiber to 3XMOyFdB (5EE7PIWY) (02/2024)
Cantidad: 942.71
Rate: 1,918.01
Amount: 1,808,127.21



In [4]:
import pandas as pd
import numpy as np

data = '''Transport | Switch Fiber Pair (Intra-campus) | Pairs (4419693704) (04/2023|10 Gbps Fiber\t2,912.98\t934.09\t2,720,985.49
to HOEpyb (YSPG4VFH) (04/2023)
Carrier Taxes for Transport | 230 Gbps Wavelength Diverse between Sparks, OR 56789 and\t3,500.87\t6,229.33\t21,808,074.52
Plano, NV 98765 (SNpTfT) (NJYM5MQP) (07/2023 Taxes) (07/2023)
Special Partnership Discount (03/2023)\t\t\t\t\t8.00\t-561.00\t-4,488.00
Item Discount: Special Partnership Discount (11/2023)\t\t\t\t8.00\t-296.00\t-2,368.00
Transport | 82 Gbps Fiber to YDDTJOrnuW (L0KSKNCB) (01/2024)\t\t\t5,607.34\t656.07\t3,678,807.55
Carrier Taxes for Transport | 45 Gbps Wavelength Diverse between Hillsboro, NV 98765\t3,949.46\t7,128.95\t28,155,502.87
and Portland, TX 56789 (a488ZH) (8D3PKG77) (05/2023 Taxes) (05/2023)
Transport | 97 Gbps Fiber to 3XMOyFdB (5EE7PIWY) (02/2024)\t\t\t942.71\t1,918.01\t1,808,127.21'''

# Separar líneas
lines = data.split("\n")

rows = []
current_item = ""

for line in lines:
    parts = line.split("\t")
    if len(parts) > 1 and any(char.isdigit() for char in parts[-1]):
        if current_item:
            rows.append([current_item] + parts)
            current_item = ""
        else:
            rows.append(parts)
    else:
        current_item += " " + line.strip() if current_item else line.strip()

# Determinar el número máximo de columnas en las filas procesadas
max_columns = max(len(row) for row in rows)

# Normalizar filas agregando NaN donde falten columnas
for row in rows:
    while len(row) < max_columns:
        row.append(np.nan)

# Crear DataFrame con nombres de columnas dinámicos
column_names = ["Descripción"] + [f"Precio {i}" for i in range(1, max_columns)]
df = pd.DataFrame(rows, columns=column_names)

df


,Descripción,Precio 1,Precio 2,Precio 3,Precio 4,Precio 5,Precio 6,Precio 7,Precio 8
0,Transport | Switch Fiber Pair (Intra-campus) |...,"2,912.98",934.09,"2,720,985.49",NaN,NaN,NaN,NaN,NaN
1,to HOEpyb (YSPG4VFH) (04/2023),Carrier Taxes for Transport | 230 Gbps Wavelen...,"3,500.87","6,229.33","21,808,074.52",NaN,NaN,NaN,NaN
2,"Plano, NV 98765 (SNpTfT) (NJYM5MQP) (07/2023 T...",Special Partnership Discount (03/2023),,,,,8.00,-561.00,"-4,488.00"
3,Item Discount: Special Partnership Discount (1...,,,,8.00,-296.00,"-2,368.00",NaN,NaN
4,Transport | 82 Gbps Fiber to YDDTJOrnuW (L0KSK...,,,"5,607.34",656.07,"3,678,807.55",NaN,NaN,NaN
5,Carrier Taxes for Transport | 45 Gbps Waveleng...,"3,949.46","7,128.95","28,155,502.87",NaN,NaN,NaN,NaN,NaN
6,"and Portland, TX 56789 (a488ZH) (8D3PKG77) (05...",Transport | 97 Gbps Fiber to 3XMOyFdB (5EE7PIW...,,,942.71,"1,918.01","1,808,127.21",NaN,NaN


In [9]:
import pandas as pd

# Cadena de datos tal como llega:
data = ('Transport | Switch Fiber Pair (Intra-campus) | Pairs (4419693704) (04/2023|10 Gbps Fiber\t2,912.98\t934.09\t2,720,985.49\n'
        'to HOEpyb (YSPG4VFH) (04/2023)\n'
        'Carrier Taxes for Transport | 230 Gbps Wavelength Diverse between Sparks, OR 56789 and\t3,500.87\t6,229.33\t21,808,074.52\n'
        'Plano, NV 98765 (SNpTfT) (NJYM5MQP) (07/2023 Taxes) (07/2023)\n'
        'Special Partnership Discount (03/2023)\t\t\t\t\t8.00\t-561.00\t-4,488.00\n'
        'Item Discount: Special Partnership Discount (11/2023)\t\t\t\t8.00\t-296.00\t-2,368.00\n'
        'Transport | 82 Gbps Fiber to YDDTJOrnuW (L0KSKNCB) (01/2024)\t\t\t5,607.34\t656.07\t3,678,807.55\n'
        'Carrier Taxes for Transport | 45 Gbps Wavelength Diverse between Hillsboro, NV 98765\t3,949.46\t7,128.95\t28,155,502.87\n'
        'and Portland, TX 56789 (a488ZH) (8D3PKG77) (05/2023 Taxes) (05/2023)\n'
        'Transport | 97 Gbps Fiber to 3XMOyFdB (5EE7PIWY) (02/2024)\t\t\t942.71\t1,918.01\t1,808,127.21')

# La idea es recorrer cada línea y reconstruir cada registro.
# Cuando la línea contiene un tabulador ("\t") se asume que comienza un nuevo registro;
# si la línea NO contiene tabulador se entiende que es una continuación de la descripción del registro anterior.

rows = []         # Aquí se almacenarán las filas (cada una será una lista con 4 elementos)
current_row = None

# Se separa el string en líneas
for line in data.splitlines():
    if "\t" in line:
        # Si ya había un registro en construcción, lo guardamos
        if current_row is not None:
            rows.append(current_row)
        # Separamos la línea por tabuladores
        tokens = line.split("\t")
        # Suponemos que los últimos 3 tokens son los valores numéricos (columna 2,3 y 4)
        # y que lo que precede forma la descripción.
        # Es posible que haya tokens vacíos dentro de la parte de la descripción, por lo que se filtran.
        description_tokens = tokens[:-3]
        description = " ".join(tok.strip() for tok in description_tokens if tok.strip())
        # Tomamos los tres últimos tokens (limpiando espacios)
        col1 = tokens[-3].strip()
        col2 = tokens[-2].strip()
        col3 = tokens[-1].strip()
        current_row = [description, col1, col2, col3]
    else:
        # La línea no tiene tabulador: es continuación de la descripción
        if current_row is not None:
            current_row[0] = current_row[0] + " " + line.strip()

# Agregamos el último registro si existe
if current_row is not None:
    rows.append(current_row)

# Creamos el DataFrame asignando nombres a las columnas (pueden llamarse como se requiera)
df = pd.DataFrame(rows, columns=["Descripción", "Col1", "Col2", "Col3"])

# Veamos el resultado:
#print(df)
print("\nLa descripción de la primera fila es:")
df.iloc[0,0]



La descripción de la primera fila es:


'Transport | Switch Fiber Pair (Intra-campus) | Pairs (4419693704) (04/2023|10 Gbps Fiber to HOEpyb (YSPG4VFH) (04/2023)'